# Comparation & evaluation of recommender systems 

In [27]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pickle
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from numpy import mean,sqrt,square
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import auxiliar_functions as af
from datetime import timedelta
import plotly.figure_factory as ff

In [2]:
variables_interes=['pyrite law', 'chalcopyrite law',
       'chalcocite law', 'covelin law', 'crusher index', 'sag power index',
       'ball work index', 'bornite law','charge cell', 'speed','HH TPH', 'granulometry','Edad','loss of TPH',"solid percentage","HH charge cell","LL charge cell","TPH",
       #"recommendation base model carl","recommendation base model acn",
       "water","power"]

In [3]:
# read df full csv
df_features=pd.read_csv('../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')[variables_interes]
df_features.dropna(inplace=True)
df_features.sort_index(inplace=True)
df_features.shape

(859344, 20)

In [4]:
# Test
df_features=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
df_features.shape

(133995, 20)

# Recommendation base model ACN

In [5]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//models//pickles//Modelo update Accenture//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//models//pickles//Modelo update Accenture//cc_recommendations_prod.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK['s'+subcontexto].loc[granulometria]["cc"]
  
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [6]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [7]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model acn"]=Rec_HH_CC

# Recommendation base model Carl

In [8]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//models//pickles//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//models//pickles//Curvas de celda de carga vs granulometria productivo actualizado.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK[subcontexto].loc[granulometria]["cc"]
    
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [9]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [10]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model carl"]=Rec_HH_CC

# Recommendation ML model

In [11]:
df_rec_ml_model=pd.read_csv('../../data/processed data/recommendation_30March2023.csv',parse_dates=["Timestamp"],index_col="Timestamp").rename(columns={"start loss tph":"predict start loss tph"}).drop(columns=["loss of TPH"])
print(df_rec_ml_model.shape)
df_rec_ml_model.head()

(133995, 4)


,recommended HH charge cell,TPH optimum,TPH predict,predict start loss tph
Timestamp,,,,
2020-04-01 00:00:00,820.0,4097.7637,4046.4214,0
2020-04-01 00:01:00,820.0,4098.3223,4063.1897,0
2020-04-01 00:02:00,820.0,4168.7230,4041.3662,0
2020-04-01 00:03:00,820.0,4168.7230,4041.3662,0
2020-04-01 00:04:00,820.0,4127.3790,4054.4438,0


In [32]:
# Df para evaluar con metrica
df_merge=df_features.merge(df_rec_ml_model, left_index=True, right_index=True,how='left',suffixes=('_x', '_y'))
df_merge.head()

,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,charge cell,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,TPH,water,power,cluster,recommendation base model acn,recommendation base model carl,recommended HH charge cell,TPH optimum,TPH predict,predict start loss tph
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-01 00:00:00,2.84,1.53,0.54,0.03,20.9675,116.3165,17.0169,0.02,755.748750,9.249551,3800.0,30.530144,2.0,0,65.032940,840.0,760.0,4139.451400,2227.622872,23865.269824,0,797.969046,836.949173,820.0,4097.7637,4046.4214,0
2020-04-01 00:01:00,2.84,1.53,0.54,0.03,20.9675,116.3165,17.0169,0.02,753.892990,9.249661,3800.0,30.514721,2.0,0,65.079082,840.0,760.0,4167.058761,2237.492831,23843.675060,0,797.969046,836.949173,820.0,4098.3223,4063.1897,0
2020-04-01 00:02:00,2.84,1.53,0.54,0.03,20.9675,116.3165,17.0169,0.02,752.232083,9.249387,3800.0,30.583668,2.0,0,65.027029,840.0,760.0,4172.829661,2244.134844,23779.315798,0,797.969046,836.949173,820.0,4168.7230,4041.3662,0
2020-04-01 00:03:00,2.84,1.53,0.54,0.03,20.9675,116.3165,17.0169,0.02,749.863432,9.250801,3800.0,30.753843,2.0,0,65.059235,840.0,760.0,4176.842600,2243.161056,23740.527706,0,797.969046,836.949173,820.0,4168.7230,4041.3662,0
2020-04-01 00:04:00,2.84,1.53,0.54,0.03,20.9675,116.3165,17.0169,0.02,748.145028,9.250322,3800.0,30.850665,2.0,0,64.927029,840.0,760.0,4157.952353,2245.406316,23710.699931,0,797.969046,836.949173,820.0,4127.3790,4054.4438,0


In [33]:
# df para generar comparación de distribución y df para visualización
df_merge_distribution_recommendation=df_merge.copy()
df_merge_visualization_recommendation=df_merge.copy()

# Distribution recommendations

In [34]:
# Generación de delta TPH y loss TPH 
df_merge_distribution_recommendation["delta HH TPH"]=df_merge_distribution_recommendation["HH TPH"]-df_merge_distribution_recommendation["TPH"]
df_merge_distribution_recommendation["loss of TPH"]=df_merge_distribution_recommendation["delta HH TPH"].apply(lambda x: 1 if x>100 else 0)

# Información de perdida de TPH en los proximos 5 minutos
for i in range(1,6):
    df_merge_distribution_recommendation[f"loss of TPH_{i}"]=df_merge_distribution_recommendation["loss of TPH"].shift(i)

# Suavización del estado de perdida
df_merge_distribution_recommendation["loss of TPH"]=df_merge_distribution_recommendation.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or 
                                                     (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)

# Inicio de la perdida de tph
df_merge_distribution_recommendation["start loss tph"]=af.loss_tph(df_merge_distribution_recommendation,"loss of TPH")["start loss tph"].to_list()

In [35]:
# Desfase de 10 minuto hacia el pasado del inicio de la caida de TPH, esto para poder visualizar que valor de HH CC se recomendaba 10 minutos antes del inicio de la caida
df_merge_distribution_recommendation["start loss tph"]=df_merge_distribution_recommendation["start loss tph"].shift(-10)
df_merge_distribution_recommendation["start loss tph"].dropna(inplace=True)

In [36]:
# Filtro para escenarios de inicio de caida
df_merge_distribution_recommendation=df_merge_distribution_recommendation[df_merge_distribution_recommendation["start loss tph"]==1]
df_merge_distribution_recommendation.head(2)

,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,charge cell,speed,HH TPH,granulometry,Edad,loss of TPH,solid percentage,HH charge cell,LL charge cell,TPH,water,power,cluster,recommendation base model acn,recommendation base model carl,recommended HH charge cell,TPH optimum,TPH predict,predict start loss tph,delta HH TPH,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-01 00:46:00,2.84,1.53,0.54,0.03,20.9675,116.3165,17.0169,0.02,791.153357,9.249962,3800.0,29.751123,2.0,0,65.329404,840.0,760.0,4224.924292,2241.905138,24697.770182,0,801.548784,834.937551,820.0,3828.0151,3834.233,0,-424.924292,0.0,0.0,0.0,0.0,0.0,1.0,0,0
2020-04-01 03:20:00,2.59,1.43,0.34,0.02,20.0698,117.8203,16.9550,0.01,763.515425,9.249693,3800.0,29.720260,2.0,0,64.959508,840.0,760.0,4174.167145,2250.330616,24154.360853,0,801.548784,834.937551,820.0,3963.8296,3948.786,0,-374.167145,0.0,0.0,0.0,0.0,0.0,1.0,0,0


In [37]:
df_merge_distribution_recommendation[["recommendation base model acn","recommendation base model carl","recommended HH charge cell"]].describe()

,recommendation base model acn,recommendation base model carl,recommended HH charge cell
count,706.000000,706.000000,706.000000
mean,829.341047,852.942280,791.770538
std,40.623121,48.552816,36.933654
min,739.940776,741.707635,750.000000
25%,790.438998,816.832953,770.000000
50%,845.822493,859.318138,770.000000
75%,866.139850,892.604103,810.000000
max,889.411577,929.011873,890.000000


In [38]:
x1 = df_merge_distribution_recommendation["recommended HH charge cell"]
x2 = df_merge_distribution_recommendation["recommendation base model carl"]
x3 = df_merge_distribution_recommendation["recommendation base model acn"]

fig = go.Figure()
fig.add_trace(go.Box(y=x1,  name="recommended HH charge cell"))
fig.add_trace(go.Box(y=x2 , name="recommendation base model carl"))
fig.add_trace(go.Box(y=x3, name="recommendation base model acn"))
fig.update_traces(boxpoints='all')
fig.update_layout(height=600, width=1200, title_text="Box plot of recommendations")
fig.show()

In [39]:
hist_data = [x1]

group_labels = ['recommended HH charge cell']
colors = ['#A56CC1']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels,# colors=colors,
                         #bin_size=.2, 
                         show_rug=False,
                         show_hist=False
                         )

# Add title
fig.update_layout(title_text='Hist and Curve Plot')
fig.update_layout(height=600, width=1200, title_text="Distribution of recommendations")
fig.show()

In [40]:
hist_data = [x1]

group_labels = ['recommended HH charge cell']
colors = ['#A56CC1']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels,# colors=colors,
                         #bin_size=.2, 
                         show_rug=False,
                         show_hist=True
                         )

# Add title
fig.update_layout(title_text='Hist and Curve Plot')
fig.update_layout(height=600, width=1200, title_text="Distribution of recommendations")
fig.show()

In [41]:
hist_data = [x1,x2,x3]

group_labels = ['recommended HH charge cell',"recommendation base model carl","recommendation base model acn"]
colors = ['#A56CC1']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels,# colors=colors,
                         #bin_size=.2, 
                         show_rug=False,
                         show_hist=False)
# Add title
fig.update_layout(title_text='Hist and Curve Plot')
fig.update_layout(height=600, width=1200, title_text="Distribution of recommendations")
fig.show()

In [42]:
hist_data = [x1,x2,x3]

group_labels = ['recommended HH charge cell',"recommendation base model carl","recommendation base model acn"]
colors = ['#A56CC1']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels,# colors=colors,
                         #bin_size=.2, 
                         show_rug=False,
                         show_hist=True)
# Add title
fig.update_layout(title_text='Hist and Curve Plot')
fig.update_layout(height=600, width=1200, title_text="Distribution of recommendations")
fig.show()

# Visualization

In [44]:
# Generación de delta TPH y loss TPH 
df_merge_visualization_recommendation["delta HH TPH"]=df_merge_visualization_recommendation["HH TPH"]-df_merge_visualization_recommendation["TPH"]
df_merge_visualization_recommendation["loss of TPH"]=df_merge_visualization_recommendation["delta HH TPH"].apply(lambda x: 1 if x>100 else 0)

# Información de perdida de TPH en los proximos 5 minutos
for i in range(1,6):
    df_merge_visualization_recommendation[f"loss of TPH_{i}"]=df_merge_visualization_recommendation["loss of TPH"].shift(i)

# Suavización del estado de perdida
df_merge_visualization_recommendation["loss of TPH"]=df_merge_visualization_recommendation.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or 
                                                     (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)

# Inicio de la perdida de tph
df_merge_visualization_recommendation["start loss tph"]=af.loss_tph(df_merge_visualization_recommendation,"loss of TPH")["start loss tph"].to_list()

In [ ]:
titulos=tuple(["TPH & HH TPH","charge cell","granulometry & SPI","speed",'water',"solid percentage","start loss tph","loss of TPH"])
fig = make_subplots(
    rows=8, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["TPH optimum"],
                    mode='lines',
                    name='TPH predict opt'),row=1, col=1)

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["TPH predict"],
                    mode='lines',
                    name='TPH predict'),row=1, col=1)
fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["HH TPH"],
                    mode='lines',
                    name='HH TPH'),row=1, col=1)

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["TPH"],
                    mode='lines',
                    name='TPH'),row=1, col=1)

##
fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["HH charge cell"],
                    mode='lines',
                    name='HH charge cell'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['recommendation base model acn'],
                    mode='lines',
                    name='recommendation base model acn'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['recommendation base model carl'],
                    mode='lines',
                    name='recommendation base model carl'),row=2, col=1)


fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["recommended HH charge cell"],
                    mode='lines',
                    name='recommended HH charge cell ML model'),row=2, col=1)


fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["LL charge cell"],
                    mode='lines',
                    name='LL charge cell'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["charge cell"],
                    mode='lines',
                    name='charge cell'),row=2, col=1)  


fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,)),row=3, col=1)  


fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, )),row=3, col=1)  


fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, )),row=4, col=1)  

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, )),row=5, col=1)  

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation['water'], ##FF6511
                    mode='lines',
                    name='water',line=dict(width=3, )),row=6, col=1)  


fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["start loss tph"],
                    mode='lines',
                    name="start loss TPH"),row=7, col=1)

fig.add_trace(go.Scatter(x=df_merge_visualization_recommendation.index, y=df_merge_visualization_recommendation["loss of TPH"],
                    mode='lines',
                    name="loss of TPH"),row=8, col=1)


fig.update_layout(height=2000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")

fig.show()

# Evaluation

In [14]:
sistem_recommender='recommendation base model carl'

def metric_system_recommendation(sistem_recommender,df1):

    df1["dif_TPH_HH"]=df1["HH TPH"]-df1["TPH"]
    df1["dif_CC_HH"]=df1["HH charge cell"]-df1["charge cell"]
    df1["dif_CC_LL"]=df1["charge cell"]-df1["LL charge cell"]

    # Información de perdida de TPH en los proximos 5 minutos
    for i in range(1,6):
        df1[f"loss of TPH_{i}"]=df1["loss of TPH"].shift(i)

    df1["greater than HH CC"]=df1["dif_CC_HH"].apply(lambda x: 0 if x>1 else 1)
    for i in range(1,6):
        df1[f"greater than HH CC_{i}"]=df1["greater than HH CC"].shift(i)

    df1["less than LL CC"]=df1["dif_CC_LL"].apply(lambda x: 0 if x>1 else 1)
    for i in range(1,6):
        df1[f"less than LL CC_{i}"]=df1["less than LL CC"].shift(i)

    df1["greater than Rec HH CC"]=df1.apply(lambda x: 1 if x[sistem_recommender]<=x["charge cell"] else 0,axis=1)
    for i in range(1,6):
        df1[f"greater than Rec HH CC_{i}"]=df1["greater than Rec HH CC"].shift(i)

    df1["Rec HH CC less than HH CC"]=df1.apply(lambda x: 1 if x[sistem_recommender]<x["HH charge cell"] else 0,axis=1)
    for i in range(1,6):
        df1[f"Rec HH CC less than HH CC_{i}"]=df1["Rec HH CC less than HH CC"].shift(i)

    df1.dropna(inplace=True)
    df1["loss of TPH"]=df1.apply(lambda x: 1 if (x['loss of TPH']==1) or (x['loss of TPH_1']==1) or (x['loss of TPH_2']==1) or (x['loss of TPH_3']==1) or (x['loss of TPH_4']==1) or (x['loss of TPH_5']==1) else 0,axis=1)
    df1=af.loss_tph(df1,"loss of TPH")

    df_full=df1.copy()

    list_df=[]
    for i in df_full[df_full["start loss tph"]==1].index:
        try:
            #i="2022-02-27 12:31:00"
            inicio_perdida=i
            test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
            fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
            inicio_evento=inicio_perdida-timedelta(minutes=5)
            data=df_full.copy()[inicio_evento:fin_perdida]
            #display(data)
            dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
            "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
            "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
            list_df.append(pd.DataFrame(dic_df,index=[1]))  
        except:
            #print(i)
            pass

    df_tph_loss=pd.concat(list_df).reset_index(drop=True)

    # Perdida de TPH por superar recomendación [+]
    evento1=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) \
        & (df_tph_loss["Supera recomendación del modelo"]==True)\
        & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
    list_loss_tph=[]
    for i in range(len(evento1)):
        data=df_full[evento1.iloc[i]["inicio perdida"]:evento1.iloc[i]["fin perdida"]]
        list_loss_tph.append(data["dif_TPH_HH"].sum())
    tph1=sum(list_loss_tph)

    # Perdida de TPH sin superar recomendación [-]
    evento2=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==False)\
        & (df_tph_loss["Supera recomendación del modelo"]==False)\
        ].reset_index(drop=True)
    list_loss_tph=[]
    for i in range(len(evento2)):
        data=df_full[evento2.iloc[i]["inicio perdida"]:evento2.iloc[i]["fin perdida"]]
        list_loss_tph.append(data["dif_TPH_HH"].sum())
    tph2=sum(list_loss_tph)
    print(tph1)
    print(tph2)
    metric_final=tph1-tph2
    
    return metric_final


In [15]:
sistem_recommender='recommendation base model carl'
metric_system_recommendation(sistem_recommender,df_merge)

126805.99292403675
273004.3074251988


-146198.31450116204

In [16]:
sistem_recommender="recommendation base model acn"
metric_system_recommendation(sistem_recommender,df_merge)

150727.46301093284
194944.60854358945


-44217.14553265661

In [17]:
sistem_recommender="recommended HH charge cell"
metric_system_recommendation(sistem_recommender,df_merge)

154504.54719049192
98972.76690049091


55531.780290001014